In [437]:
import pandas as pd
import numpy as np
import sys
import math
import matplotlib.pyplot as plt
import re
import string
from ipywidgets import IntProgress

In [438]:
# прогресс бар
def log_progress(sequence, every=1):
    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record 

# удаление пунктуации       
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    return str(s)        

Feature toggles:

In [439]:
remove_versions = False
remove_non_sim = True
remove_vendors = False
tac_sim_rule = True
remove_old = True

Параметры:

In [440]:
min_release_year = 2011
tac_length = 8

Читаем каталог девайсов со следующими признаками: 'tac', 'model_name', 'vendor_name', 'sim_count', 'release_date':

In [441]:
dev_features = ['TAC', 'MODEL_NAME', 'VENDOR_NAME', 'SIM_COUNT', 'RELEASE_DATE']
df = pd.read_csv('data/devices_dirty.csv', usecols=dev_features) 

Число записей:

In [442]:
dim_before = int(df.shape[0])
print('Число записей:', int(df.shape[0]))

Число записей: 165963


Переименовываем столбцы:

In [443]:
column_voc = {'TAC':'tac_d', 'VENDOR_NAME':'ven_d', 'MODEL_NAME':'model_d', 'SIM_COUNT':'sim_d', 'RELEASE_DATE':'date_d'}
df = df.rename(columns=column_voc)

Выводим структуру:

In [444]:
df = df[['ven_d', 'model_d', 'tac_d', 'sim_d','date_d']]
df.head(0)

,ven_d,model_d,tac_d,sim_d,date_d


Сброс записей с пропущенными значениями в ключевых атрибутах:

In [445]:
df = df.dropna(subset = ['ven_d', 'model_d', 'tac_d', 'date_d'], how='any', axis = 0)

Преобразование типов:

In [446]:
df = df.astype({'ven_d': 'str', 'model_d': 'str', 'tac_d': 'str'})
df['sim_d'] = df['sim_d'].fillna(-1).astype(int).replace(-1, np.nan)
df['date_d'] = pd.to_datetime(df['date_d']).dt.year.astype(int)

Выводим число нулей:

In [447]:
print('Число нулей для всех признаков:', dict(df.isnull().sum()))

Число нулей для всех признаков: {'ven_d': 0, 'model_d': 0, 'tac_d': 0, 'sim_d': 108295, 'date_d': 0}


Удаляем множественные пробелы:

In [448]:
df = df.applymap(lambda x: re.sub(' +', ' ', x) if isinstance(x, str) else x)

Удаляем пробелы справа и слева, приводим к нижнему регистру:

In [449]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

Удаляем из названий моделей производителей:

In [450]:
if remove_vendors is True:
    for item in log_progress(list(set(df['ven_d'])), every=1):
        df['model_d'] = df['model_d'].apply(lambda x: x.replace(item, ''))

Если в скобках указано перечисление, то удаляется содержимое скобок:

In [451]:
if remove_versions is True:
    pattern = r'\(.*?\,.*?\)'
    df['model_d'] = df['model_d'].apply(lambda x: re.sub(pattern, '', x))

Удаляем знаки пунктуации из строковых признаков:

In [452]:
df[['model_d','ven_d','tac_d']] = df[['model_d','ven_d','tac_d']].applymap(remove_punctuation)

Удаляем записи без названия модели после очистки:

In [453]:
df = df.dropna(subset = ['model_d'], how = 'any')

Создаем уникальный признак 'ven_model_d':  

In [454]:
df['ven_model_d'] = df['ven_d'] + ' ' + df['model_d']
df = df[['ven_d','model_d','ven_model_d','tac_d','sim_d','date_d']]

Создаем фрейм с датой выпуска и уникальной моделью в атрибутах:

In [455]:
df_actual = df[['ven_model_d', 'date_d']]
df_actual = df_actual.drop_duplicates()

Определяем актуальность каталога:

In [456]:
print('Мода по дате выпуска:', int(df['date_d'].mode()))
print('Минимальная дата выпуска:', int(df['date_d'].min()))
print('Максимальная дата выпуска:', int(df['date_d'].max()))

Мода по дате выпуска: 2011
Минимальная дата выпуска: 1992
Максимальная дата выпуска: 2019


Отфильтровываем записи с годом выпуска позже чем min_release_year:

In [457]:
if remove_old is True: 
    df = df[df['date_d'] >= min_release_year]

Удаляем атрибут с датой выпуска:

In [458]:
df = df.drop(columns=['date_d'])

Устройства в сим картой в %:

In [459]:
no_sim = df[df['sim_d'] > 0]['model_d'].count()/df['model_d'].count()
print('Устройства c сим картой - доля от всех моделей:', int(round(100*(1-no_sim))), '%')

Устройства c сим картой - доля от всех моделей: 66 %


Устройства с 1,2 сим картами в %:

In [460]:
sim_one = df[df['sim_d'].isin([1])]['model_d'].count()/df[df['sim_d'] > 0]['model_d'].count()
sim_two = df[df['sim_d'].isin([2])]['model_d'].count()/df[df['sim_d'] > 0]['model_d'].count()

print('Устройства c 1 sim - доля от устройств с sim картой:', int(round(100*sim_one)), '%')
print('Устройства c 2 sim - доля от устройств с sim картой:', int(round(100*sim_two)), '%')

Устройства c 1 sim - доля от устройств с sim картой: 48 %
Устройства c 2 sim - доля от устройств с sim картой: 52 %


In [461]:
df.head(0)

,ven_d,model_d,ven_model_d,tac_d,sim_d


Оставляем записи с корректной длиной tac:

In [462]:
df = df[df['tac_d'].apply(len) == tac_length]

Оставляем девайсы с sim картой:

In [463]:
df = df[df['sim_d'] >= 1]

Сбрасываем записи с нулями:

In [464]:
df.dropna(axis=0, how='any', inplace=True)
print('Число нулей для всех признаков:', dict(df.isnull().sum()))

Число нулей для всех признаков: {'ven_d': 0, 'model_d': 0, 'ven_model_d': 0, 'tac_d': 0, 'sim_d': 0}


Неоднозначность {tac => model}:

In [465]:
df_tac = df[['ven_model_d', 'tac_d']].groupby('tac_d').count()
print('На один tac приходится max', df_tac['ven_model_d'].max(), 'устройств(о)')

На один tac приходится max 1 устройств(о)


Неоднозначность {model => tac}:

In [466]:
df_mod = df[['ven_model_d', 'tac_d']].groupby('ven_model_d').count()
print('На одно устройство приходится max', round(df_mod['tac_d'].max()), 'tac(s)')

На одно устройство приходится max 160 tac(s)


Добавляем в датафрейм признак с числом tac номеров для модели:

In [467]:
df_tac_num = df[['ven_model_d', 'tac_d']].groupby('ven_model_d', as_index=False).count().rename(columns={'tac_d': 'tac_num_d'})
df = pd.merge(df, df_tac_num, on=['ven_model_d'], how='inner')

n(tacs)<=n(sims). Исключаем записи, которые нарушают данное правило: 

In [468]:
if tac_sim_rule is True:
    df = df[df['tac_num_d'] <= df['sim_d']]

Удаляем атрибут sim_d:

In [469]:
df = df.drop(columns=['sim_d'])

Показываем структуру:

In [470]:
df.head(0)

,ven_d,model_d,ven_model_d,tac_d,tac_num_d


Сброс дубликатов по tac:

In [471]:
df = df.drop_duplicates(subset=['tac_d', 'ven_model_d'], keep=False)

Число записей:

In [472]:
dim_after = int(df.shape[0])
print('Размерность данных:', int(df.shape[0]))

Размерность данных: 8510


Компрессия по числу строк:

In [473]:
print('Компрессия по числу записей:', round(dim_before/dim_after), 'раз')

Компрессия по числу записей: 20 раз


Рассчитываем мощность множества уникальных моделей:

In [474]:
print('Мощность множества моделей:', len(set(df['ven_model_d'])))

Мощность множества моделей: 7631


Сбрасываем вспомогательные признаки:

In [475]:
df = df.drop(columns=['tac_num_d', 'ven_model_d'])

In [476]:
df.head(0)

,ven_d,model_d,tac_d


Записываем датафрейм в туже папку:

In [477]:
df.to_csv('data/devices_clean.csv', index=False)